# QEを用意する

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import importlib

# Google Colaboratory上でQuantum ESPRESSO(QE)を用意する

まず自分のgoogle driveをマウントする。（google accountが必要です。）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

既にコンパイルしたQEがある場合にはコピーする。ない場合にはダウンロードしてコンパイルすると同時に、次回に向けてMy Driveにzipファイルを退避させておく。

In [ ]:
if not os.path.exists("/content/q-e/"):
    if os.path.isfile("/content/drive/MyDrive/q-e.zip"): 
        %cd /content/
        !unzip /content/drive/MyDrive/q-e.zip
    else:
        %cd /content/
        !git clone https://github.com/minoru-otani/q-e.git
        !apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc
        %cd q-e/
        !DFLAGS='-D__FFTW3 -D__MPI' FFT_LIBS='-lfftw3' ./configure
        !make pw pp
        %cd /content/
        !zip -r /content/drive/MyDrive/q-e.zip q-e

パスを設定

In [ ]:
os.environ['PATH'] = "/content/q-e/bin:"+os.environ['PATH']
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun --allow-run-as-root -np 4 pw.x -in PREFIX.pwi > PREFIX.pwo"
qef_pseudo = 'http://www.quantum-espresso.org/wp-content/uploads/upf_files/'
pseudo_dir = "/content/q-e/pseudo/"

## Docker container上でQuantum ESPRESSO(QE)を用意する

In [ ]:
!git clone https://github.com/minoru-otani/q-e.git -b qe-6.8 --depth 1
%cd q-e
!DFLAGS='-D__FFTW3 -D__MPI' FFT_LIBS='-lfftw3 -lfftw3_omp' ./configure --enable-openmp
!make pw pp neb